# C1 W4 Group 8 - Task 3


In [1]:
from PIL import Image

from src.data import AUGMENTATIONS_QSD1_W4_LIST, FRAMES_QSD1_W4_LIST, GT_QSD1_W4_LIST
from src.paths import BBDD_PATH, QSD1_W4_PATH, QSD1_NON_AUGMENTED_W4_PATH, WEEK_4_RESULTS_PATH
from tqdm import tqdm

In [2]:
BBDD_PATH_LIST = sorted(BBDD_PATH.glob("*.jpg"))
QSD1_W4_PATH_LIST = sorted(QSD1_W4_PATH.glob("*.jpg"))
QSD1_NON_AUGMENTED_W4_PATH_LIST = sorted(QSD1_NON_AUGMENTED_W4_PATH.glob("*.jpg"))

In [3]:
database_image_PIL_list = [Image.open(db_img_path) for db_img_path in BBDD_PATH_LIST]  # Load once
for idx, db_img in enumerate(database_image_PIL_list):
    assert db_img.filename.endswith(f"{idx}.jpg")

In [4]:
query_d1_image_PIL_list = [Image.open(query_img_path) for query_img_path in QSD1_W4_PATH_LIST]  # Load once
for idx, query_img in enumerate(query_d1_image_PIL_list):
    assert query_img.filename.endswith(f"{idx}.jpg")

In [5]:
non_augmented_d1_image_PIL_list = [Image.open(query_img_path) for query_img_path in QSD1_NON_AUGMENTED_W4_PATH_LIST]  # Load once
for idx, query_img in enumerate(non_augmented_d1_image_PIL_list):
    assert query_img.filename.endswith(f"{idx}.jpg")

In [6]:
for file, augmentation, frames, gt in zip(QSD1_W4_PATH_LIST, AUGMENTATIONS_QSD1_W4_LIST, FRAMES_QSD1_W4_LIST, GT_QSD1_W4_LIST):
    print(f"File={file.stem}, Aug={augmentation}, Frames={frames}, GT={gt}")

File=00000, Aug=None, Frames=[[np.float64(179.49631381503272), [[64, 41], [2048, 25], [2016, 3653], [196, 3637]]]], GT=[-1]
File=00001, Aug=None-MultiplyHue, Frames=[[np.float64(179.52154949700142), [[19, 38], [509, 37], [496, 665], [17, 661]]]], GT=[150]
File=00002, Aug=None-MultiplyHue, Frames=[[np.float64(0.9495668564358084), [[34, 50], [396, 32], [408, 470], [46, 476]]], [np.float64(0.3015539498682642), [[466, 34], [844, 20], [850, 484], [470, 486]]]], GT=[48, 251]
File=00003, Aug=UnnamedImpulseNoise, Frames=[[np.float64(180.0), [[24, 27], [458, 26], [462, 355], [45, 355]]]], GT=[32]
File=00004, Aug=None-MultiplyHue, Frames=[[np.float64(0.4841781604809512), [[53, 78], [1489, 62], [1497, 2326], [77, 2338]]]], GT=[161]
File=00005, Aug=None, Frames=[[np.float64(179.85346379900508), [[50, 43], [1642, 27], [1646, 2051], [82, 2047]]]], GT=[81]
File=00006, Aug=None, Frames=[[np.float64(1.5481576989779455), [[176, 46], [470, 34], [478, 326], [182, 334]]], [np.float64(1.3510650827196287), [

## Background Removal

In [7]:
from src.background import get_painting_masks, crop_image_by_mask

In [8]:
painting_masks_list_d1 = []
pbar = tqdm(zip(QSD1_W4_PATH_LIST, query_d1_image_PIL_list))
for name, image in pbar:
    pbar.set_description(f"Processing {name.stem}")
    painting_masks = get_painting_masks(image)
    painting_masks_list_d1.append(painting_masks)

Processing 00029: : 30it [00:17,  1.70it/s]


In [9]:
cropped_query_image_list_d1 = []
pbar = tqdm(zip(QSD1_W4_PATH_LIST, query_d1_image_PIL_list, painting_masks_list_d1))
for name, image, predicted_mask in pbar:
    pbar.set_description(f"Splitting {name.stem}")
    image_list = crop_image_by_mask(image, predicted_mask)
    cropped_query_image_list_d1.append(image_list)

Splitting 00029: : 30it [00:00, 33.61it/s]


## Image Denoising

In [10]:
from src.denoising import denoise_image

In [11]:
denoised_query_d1_PIL_list = []
for image_tuple in cropped_query_image_list_d1:
    temp = []
    for img in image_tuple:
        temp.append(denoise_image(img))
    denoised_query_d1_PIL_list.append(temp)

## Retrieval

In [12]:
from src.retrieval import find_top_k_similar_images

In [13]:
retrieved_images_d1 = find_top_k_similar_images(denoised_query_d1_PIL_list, database_image_PIL_list, k=10)

Processing level 5: 100%|██████████| 36/36 [00:14<00:00,  2.47it/s]


In [14]:
print(retrieved_images_d1)

[[[270, 47, 248, 24, 189, 40, 120, 101, 279, 102]], [[150, 215, 51, 4, 70, 212, 95, 40, 90, 132]], [[48, 116, 201, 40, 22, 184, 258, 212, 120, 224], [251, 234, 119, 177, 182, 47, 189, 100, 120, 137]], [[32, 193, 104, 257, 130, 242, 161, 88, 31, 103]], [[161, 40, 116, 35, 70, 132, 120, 274, 106, 68]], [[81, 163, 226, 205, 60, 140, 252, 244, 96, 248]], [[176, 237, 58, 145, 233, 239, 43, 114, 126, 150]], [[270, 101, 248, 60, 47, 40, 279, 258, 24, 91]], [[264, 99, 104, 279, 259, 113, 32, 91, 242, 128]], [[269, 184, 228, 92, 90, 6, 165, 167, 42, 5]], [[167, 95, 249, 228, 16, 25, 165, 90, 260, 150], [76, 142, 273, 110, 103, 104, 57, 113, 242, 140]], [[3, 206, 11, 35, 149, 40, 78, 13, 205, 61]], [[46, 101, 270, 248, 63, 90, 60, 232, 249, 258]], [[53, 54, 107, 238, 30, 74, 11, 14, 106, 164]], [[163, 205, 60, 226, 286, 251, 137, 68, 35, 240]], [[12, 265, 193, 61, 74, 161, 80, 253, 197, 238]], [[11, 222, 283, 206, 245, 238, 40, 104, 161, 255], [221, 35, 48, 205, 25, 165, 119, 232, 116, 120]], [[

## Evaluation

In [15]:
from src.metrics import MeanAveragePrecisionAtK

map = MeanAveragePrecisionAtK()

In [16]:
GT_mod = []
for element in GT_QSD1_W4_LIST:
    if len(element) == 2:
        GT_mod.append([element[0]])
        GT_mod.append([element[1]])
    else:
        GT_mod.append([element[0]])

results_mod = []
for element in retrieved_images_d1:
    for e in element:
        results_mod.append(e)

In [17]:
result = map.compute(GT_mod, results_mod, k=10)
print(f"MAP QSD1_W4 Method Week 3: {result}")

MAP QSD1_W4 Method Week 3: 0.16666666666666666
